In [ ]:
决策树
优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据
缺点：可能产生过度匹配
适用数据类型：数值型和标称型
一般流程：
1、收集数据：可以使用任何方法
2、准备数据：树构造算法只适用于标称型数据，因此数值型数据必须离散化
3、分析数据：检查在构造的树后的图形是否符合预期
4、训练算法：构造树的数据结构
5、测试算法：计算错误率
6、使用算法

In [ ]:
1、决策树的构造
1.1
   第一个问题：当前数据集哪个特征在划分数据分类时起关键作用？
   为了找到决定性的特征，划分最好的结果，我们必须评估每个特征，完成测试后原始数据被划分成了几个数据子集，这些子集分布在第一个决策点的所有分支上。如果某个分支下的数据属于同一类型，那么无需继续进行分割，否则继续分割。
   伪代码：
   检测数据集中每个子项是否属于同一分类：
     if so return 类标签
     else
       寻找划分数据集合的最好特征
       划分数据集
       创建分直接点
       for each子集
         递归调用本函数返回结果到分支节点中
      return 分支节点
1.2
   如何划分数据找到决定性的特征呢？
   划分数据集的大原则是：将无序的数据变的更加有序
   组织杂乱无章的数据的一种方法就是使用信息论度量信息，信息论是量化处理信息的分支科学，在划分数据集前后发生的变化称为信息增益，获得信息增益最高的特征就是最好的选择。
   熵定义为信息的期望值，在此之前我们需要知道信息的定义：如果待分类的事务可能划分在多个分类中，则符号x的信息定义为l(x)=-log2 p(x) 其中p(x)是选择该分类的概率
   为了计算熵，我们需要将计算的所有类别所有可能值包含的信息期望值，H=-Σp(x) log2 p(x)

In [76]:
#计算给定数据集的熵
from math import log
def calEnt(dataset):
    numEntries=len(dataset)
    labelCount={}
    for feaVec in dataset:
        curlabel=feaVec[-1]
        if curlabel not in labelCount.keys():
            labelCount[curlabel]=0
        labelCount[curlabel]+=1
    Ent=0.0
    for key in labelCount:
        p=float(labelCount[key]/numEntries)
        Ent-=p*log(p,2)
    return Ent
#测试
def createdataset():
    dataset=[[1,1,'yes'],
             [1,1,'yes'],
             [1,0,'no'],
             [0,1,'no'],
             [0,1,'no']
            ]
    labels=['no sufacing','filippers']
    return dataset,labels
mydata,mylabel=createdataset()

   上面我们学习了如何度量数据集的无序程度，分类算法除了需要测量信息熵，还需要划分数据集，度量划分数据集合的熵，以便判断当前是否正确划分了数据集，我们将对每个特征划分数据集的结果计算一次信息熵，然后判断那个特征划分是最好的划分方式。

In [78]:
#根据数据的特征以及每个特征不同取值划分数据集
def splitdatabyfeature(dataset,index,value):
    restdataset=[]
    for feature in dataset:
        if feature[index]==value:
            reducedfeature=feature[:index]
            reducedfeature.extend(feature[index+1:])
            restdataset.append(reducedfeature)
    return restdataset


#print(splitdatabyfeature(mydata,0,1))

In [79]:
#计算数据的每个特征的信息增益
#基本步骤：
#1、计算当前熵值
#2、计算每个特征对应的信息增益
#   2.1根据一个特征的不同取值划分数据集
#   2.2根据不同取值value在特征中所占比例，计算熵值
#   2.3计算信息增益 Ent(D)-Σ（prob *Ent（DV））
#3、记录并返回最大信息增益对应的特征
def chooseBestFeatureTosplit(dataset):
    numfeature=len(dataset[0])-1
    bestinfogain=0.0
    bestfeature=-1
    Ent=calEnt(dataset)
    for i in range(numfeature):
        category=[example[i] for example in dataset]
        category=set(category)#去重                             
        #print(category)
        newEnt=0.0
        for value in category:
            restdata=splitdatabyfeature(dataset,i,value)
            prob=len(restdata)/float(len(dataset))
            newEnt+=prob*calEnt(restdata)
        infogain=Ent-newEnt
        if infogain>bestinfogain:
            bestinfogain=infogain
            bestfeature=i
    return bestfeature   

#print(chooseBestFeatureTosplit(mydata))

In [80]:
import operator
def majorityCnt(classname):
    classcount={}
    for vote in classname:
        if vote not in classcount.keys():
            classcount[vote]=0
        classcount[vote]+=1
    sortedclasscount=sorted(classcount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedclasscount[0][0]
majorityCnt(['yes','yes','no','no','no'])

'no'

In [188]:
def createtree(dataset,labels):
    classname=[example[-1] for example in dataset]
    if classname.count(classname[0])==len(classname):
        return classname[0]
    if len(dataset[0])==1:
        return majorityCnt(classname)
    selectedfeature=chooseBestFeatureTosplit(dataset)
    #print(labels)
    selectedlabel=labels[selectedfeature]
    value=[example[selectedfeature] for example in dataset]
    value=set(value)
    mytree={selectedlabel:{}}
    del(labels[selectedfeature])
    for eachvalue in value:
        currdataset=splitdatabyfeature(dataset,selectedfeature,eachvalue)
        currlabels=labels[:]
        mytree[selectedlabel][eachvalue]=createtree(currdataset,currlabels)
    return mytree
label=mylabel[:]
mytree=createtree(mydata,label)

In [132]:
#使用决策树进行分类
def classify(inputtree,labels_,features):
    FeatureName=list(inputtree.keys())[0]
    FeatureValue=inputtree[FeatureName]
    #print(labels_)
    FeatureIndex=labels_.index(FeatureName)
    SecondDict=inputtree[FeatureName]    #二级字典
    for value in FeatureValue:
        if features[FeatureIndex]==value:
            if type(SecondDict[value]).__name__=='dict':
                classlabel=classify(SecondDict[value],labels_,features)
            else:
                classlabel=SecondDict[value]
    return classlabel

classify(mytree,mylabel,[1,0])

'no'

In [170]:
#在计算机中存储决策树分类器pickle序列化对象
def storetree(inputtree,filename):
    import pickle
    fw=open(filename,'wb')
    pickle.dump(inputtree,fw)
    fw.close()
def gettree(filename):
    import pickle
    fr =open(filename,'rb')
    return pickle.load(fr)

storetree(mytree,'classifystorage.txt')
gettree('classifystorage.txt')

{'no sufacing': {0: 'no', 1: {'filippers': {0: 'no', 1: 'yes'}}}}

In [181]:
#隐形眼镜类型分类
fr=open('data.txt')
lenses=[inst.strip().split('\t') for inst in fr.readlines()]
lesessLabels=['age','prescript','astigmatic','tearrate']
lensetree=createtree(lenses,lesessLabels)
lensetree

{'tearrate': {'normal': {'astigmatic': {'no': {'age': {'pre': 'soft',
      'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}},
      'young': 'soft'}},
    'yes': {'prescript': {'hyper': {'age': {'pre': 'no lenses',
        'presbyopic': 'no lenses',
        'young': 'hard'}},
      'myope': 'hard'}}}},
  'reduced': 'no lenses'}}